# 포켓몬고 챗봇 테스트

### 기본 데이터 스크래핑
- 향후 데이터 추가

In [1]:
from dotenv import load_dotenv
import logging_chorock
load_dotenv()

logging_chorock.langsmith("poketory")

LangSmith 추적을 시작합니다.
[프로젝트명]
poketory


In [2]:
url_list = [
    "https://poketory.com/pokemongo-guidebook/", # 포켓몬 GO 게임 공략 최신판
    "https://poketory.com/pokemongo-weekly-event/", # 포켓몬 GO 게임 일정 : 이번 주의 이벤트 및 보너스 일정
    "https://poketory.com/events/", # 포켓몬 GO 게임 일정 : 달력
    "https://poketory.com/pokemongo-research/", # 포켓몬 GO 최신 필드 리서치, 대발견 목록 : 리서치 보상, 대발견 포켓몬, 도구 지급 확률
    "https://poketory.com/pokemongo_egg/", # 포켓몬 GO 알 부화
    "https://poketory.com/pokemongo_current_raid/", # 포켓몬 GO 「레이드 · 맥스 배틀」 최신 정보
    "https://poketory.com/pokemongo_tier/", # 포켓몬 GO 레이드 티어, 타입별 사용률, 초보자용 추천 포켓몬 (포켓몬 순위)
    "https://poketory.com/pokemongo_moveset/", # 포켓몬 GO 추천 기술 (졸업 기술) : 추천 포켓몬, 기술 순위 (레이드, 트레이너 배틀 · GO 배틀리그)
]

In [3]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# 뉴스기사 내용을 로드합니다.
loader = WebBaseLoader(
    web_paths=url_list,
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["entry-content clearfix", "post-header"]},
        )
    ),
    header_template={
        "User_Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36",
    },
)

docs = loader.load()
print(f"문서의 수: {len(docs)}")
docs

USER_AGENT environment variable not set, consider setting it to identify your requests.


문서의 수: 8


[Document(metadata={'source': 'https://poketory.com/pokemongo-guidebook/'}, page_content='\n포켓몬 GO 게임 공략 최신판 : 최신 정보를 담은 뉴비, 복귀 유저용 종합 가이드북!\n\n\n\n\t\t\t\t2024/06/14\t\t\t\n\n\n5 댓글 \n\n\n\n\n\n\n\n\n\n Poketory.com 사이트의 포켓몬 GO 공략 최신판입니다. 오래되거나 더이상 유효하지 않은 정보는 폐기하고 모두 최신 버전으로 업데이트 했고, 다소 압축하여 전해드립니다.\n\n\n\n\n목차\nToggle\n글 갱신 내역메뉴일정, 친구 코드 공유자주 묻는 질문게임을 시작했는데 뭐부터 해야 하나요?일단 31레벨까지 빠르게 레벨업하세요.포켓몬은 일단 30레벨까지 강화하세요.초보에게 개체값이 과연 중요할까요?별의모래와 사탕은 항상 신중하게 사용하세요.이 포켓몬 쓸모 있나요? 박스 정리는 어떻게 해요?옛날 포켓몬은 버려야 하나요? (복귀 유저)매일 체육관 방어로 50코인을 모으고, 박스와 가방을 업그레이드하세요.포켓몬을 빠르고 정확하게 잡고 싶어요!이브이 100%는 뭐로 진화하나요?이 외의 질문\n글 갱신 내역\n\n\n2024-06-14 : ‘포켓몬 잡기’에 대한 메뉴를 추가했습니다.\n\n\n\n\n\n\n메뉴\n\n포켓몬 GO 최신 공략 바로가기구분키워드 : Ctrl + F 또는 찾기!🆕최신정보최신 필드 리서치 정보대발견, 얼루기, 누적 방법최신 알 부화 정보알 부화 포켓몬, 부화장치,알 부화 속도, 레벨 & 개체값,활화르바, 알 버리기? 최신 레이드 진행 정보레이드 공략최신 숍, 웹스토어 정보가방, 박스 상한,추천 구매 박스,역대 최저가⚙️기초시스템게임 설정 방법게임 설치, 계정 및 캐릭터 생성,리퍼럴 코드, 유용한 게임 설정,인터페이스포켓몬 포획, 잡는 방법바이옴, 둥지, 야생, 날씨향로, 루어모듈, 교환 등팀 (진영) 가이드진영 (팀) 선택, 비율,팀 변경 방법, 가능한 활동매일 진행 가능한

In [ ]:
print(docs[0].page_content)

### 데이터 벡터 DB 에 임베딩 저장

- 임베딩 모델 후보 : OpenAI, BGE-M3, e5 ...
- CacheBackedEmbeddings 를 통해 재계산을 피해보자

In [4]:
import os
import warnings

# 경고 무시
warnings.filterwarnings("ignore")

# ./cache/ 경로에 다운로드 받도록 설정
os.environ["HF_HOME"] = "./cache/"

In [5]:
from langchain.storage import LocalFileStore
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings

model_name = "intfloat/multilingual-e5-large-instruct"

embedding = HuggingFaceEndpointEmbeddings(
    model=model_name,
    task="feature-extraction",
    huggingfacehub_api_token=os.environ["HUGGINGFACEHUB_API_TOKEN"],
)

# OpenAI의 "text-embedding-3-small" 모델을 사용하여 임베딩을 생성합니다.
# embedding = OpenAIEmbeddings(model="text-embedding-3-small")

# 로컬 파일 저장소 설정
store = LocalFileStore("./cache/")

# 캐시를 지원하는 임베딩 생성
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embedding,
    document_embedding_cache=store,
    namespace=embedding.model,  # 기본 임베딩과 저장소를 사용하여 캐시 지원 임베딩을 생성
)


In [6]:
from langchain_text_splitters import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 문서 로드
# raw_documents = TextLoader("./data/appendix-keywords.txt").load()
# 문자 단위로 텍스트 분할 설정
# text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

# 문서 분할
documents = text_splitter.split_documents(docs)

Created a chunk of size 2357, which is longer than the specified 1000
Created a chunk of size 1853, which is longer than the specified 1000
Created a chunk of size 1014, which is longer than the specified 1000
Created a chunk of size 1091, which is longer than the specified 1000
Created a chunk of size 1056, which is longer than the specified 1000
Created a chunk of size 1021, which is longer than the specified 1000
Created a chunk of size 1304, which is longer than the specified 1000
Created a chunk of size 1170, which is longer than the specified 1000
Created a chunk of size 1131, which is longer than the specified 1000
Created a chunk of size 3168, which is longer than the specified 1000
Created a chunk of size 2534, which is longer than the specified 1000
Created a chunk of size 1610, which is longer than the specified 1000
Created a chunk of size 2350, which is longer than the specified 1000
Created a chunk of size 2246, which is longer than the specified 1000
Created a chunk of s

In [7]:
# 코드 실행 시간을 측정합니다.
%time 
db = FAISS.from_documents(documents, cached_embedder)  # 문서로부터 FAISS 데이터베이스 생성

CPU times: total: 0 ns
Wall time: 0 ns


In [10]:
len(documents)

71

In [9]:
# 샘플 데이터
texts = [
    "이번주 레이드 알려줘",
    "다음주 메가레이드는 뭐야?",
    "Retrieval-Augmented Generation (RAG) is an effective technique for improving AI responses.",
]

In [13]:
%%time
# Document Embedding 수행
embedded_documents = embedding.embed_documents(texts)

CPU times: total: 15.6 ms
Wall time: 1.22 s


In [15]:
print("[HuggingFace Endpoint Embedding]")
print(f"Model: \t\t{model_name}")
print(f"Dimension: \t{len(embedded_documents[0])}")

[HuggingFace Endpoint Embedding]
Model: 		intfloat/multilingual-e5-large-instruct
Dimension: 	1024


In [19]:
import numpy as np
test_query = docs[0].page_content
embedded_query = embedding.embed_query(test_query)
embedded_documents = embedding.embed_documents(texts)

# 질문(embedded_query): LangChain 에 대해서 알려주세요.
np.array(embedded_query) @ np.array(embedded_documents).T

sorted_idx = (np.array(embedded_query) @ np.array(embedded_documents).T).argsort()[::-1]

# print(f"{test_query}\n====================================")
for i, idx in enumerate(sorted_idx):
    print(f"[{i}] {docs[idx].metadata}")
    print()

[0] {'source': 'https://poketory.com/pokemongo-guidebook/'}

[1] {'source': 'https://poketory.com/pokemongo-weekly-event/'}

[2] {'source': 'https://poketory.com/events/'}



In [24]:
docs

[Document(metadata={'source': 'https://poketory.com/pokemongo-guidebook/'}, page_content='\n포켓몬 GO 게임 공략 최신판 : 최신 정보를 담은 뉴비, 복귀 유저용 종합 가이드북!\n\n\n\n\t\t\t\t2024/06/14\t\t\t\n\n\n5 댓글 \n\n\n\n\n\n\n\n\n\n Poketory.com 사이트의 포켓몬 GO 공략 최신판입니다. 오래되거나 더이상 유효하지 않은 정보는 폐기하고 모두 최신 버전으로 업데이트 했고, 다소 압축하여 전해드립니다.\n\n\n\n\n목차\nToggle\n글 갱신 내역메뉴일정, 친구 코드 공유자주 묻는 질문게임을 시작했는데 뭐부터 해야 하나요?일단 31레벨까지 빠르게 레벨업하세요.포켓몬은 일단 30레벨까지 강화하세요.초보에게 개체값이 과연 중요할까요?별의모래와 사탕은 항상 신중하게 사용하세요.이 포켓몬 쓸모 있나요? 박스 정리는 어떻게 해요?옛날 포켓몬은 버려야 하나요? (복귀 유저)매일 체육관 방어로 50코인을 모으고, 박스와 가방을 업그레이드하세요.포켓몬을 빠르고 정확하게 잡고 싶어요!이브이 100%는 뭐로 진화하나요?이 외의 질문\n글 갱신 내역\n\n\n2024-06-14 : ‘포켓몬 잡기’에 대한 메뉴를 추가했습니다.\n\n\n\n\n\n\n메뉴\n\n포켓몬 GO 최신 공략 바로가기구분키워드 : Ctrl + F 또는 찾기!🆕최신정보최신 필드 리서치 정보대발견, 얼루기, 누적 방법최신 알 부화 정보알 부화 포켓몬, 부화장치,알 부화 속도, 레벨 & 개체값,활화르바, 알 버리기? 최신 레이드 진행 정보레이드 공략최신 숍, 웹스토어 정보가방, 박스 상한,추천 구매 박스,역대 최저가⚙️기초시스템게임 설정 방법게임 설치, 계정 및 캐릭터 생성,리퍼럴 코드, 유용한 게임 설정,인터페이스포켓몬 포획, 잡는 방법바이옴, 둥지, 야생, 날씨향로, 루어모듈, 교환 등팀 (진영) 가이드진영 (팀) 선택, 비율,팀 변경 방법, 가능한 활동매일 진행 가능한

### 벡터DB에 저장
- Chroma, FAISS 같은 local db 를 쓸거임

In [26]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_chroma import Chroma
from langchain_community.vectorstores import FAISS
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore

# # DB 생성
# db = Chroma.from_documents(
#     documents=documents, embedding=OpenAIEmbeddings(), collection_name="my_db"
# )

In [32]:
# 저장할 경로 지정
DB_PATH = "./chroma_db"

# 문서를 디스크에 저장합니다. 저장시 persist_directory에 저장할 경로를 지정합니다.
# persist_db = Chroma.from_documents(
#     documents, OpenAIEmbeddings(), persist_directory=DB_PATH, collection_name="my_db"
# )
persist_db = Chroma.from_documents(
    documents, embedding, persist_directory=DB_PATH, collection_name="my_db"
)

In [37]:
# 디스크에서 문서를 로드합니다.
persist_db = Chroma(
    persist_directory=DB_PATH,
    embedding_function=OpenAIEmbeddings(), # OpenAIEmbeddings(), embedding = HF
    collection_name="my_db",
)

In [35]:
# persist_db.get()

### 유사도 확인

In [39]:
persist_db.similarity_search("이번주 메가레이드 알려줘")

[Document(id='c47dd8fb-82c3-4fb3-bf19-25c3fb3c89b4', metadata={'source': 'https://poketory.com/pokemongo_current_raid/'}, page_content='메가 · 원시 레이드 공략메가이상해꽃✨메가리자몽 X✨메가리자몽 Y✨메가거북왕✨메가독침붕✨메가피죤투✨메가후딘✨메가야도란✨메가팬텀✨메가캥카✨메가쁘사이저✨메가갸라도스✨ 메가프테라✨메가뮤츠 X✨메가뮤츠 Y✨메가전룡✨메가강철톤✨메가핫삼✨메가헤라크로스✨메가헬가✨메가마기라스✨메가나무킹✨메가번치코✨메가대짱이✨메가가디안✨메가깜까미✨메가입치트✨메가보스로라✨메가요가램✨메가썬더볼트✨메가샤크니아✨메가폭타✨메가파비코리✨메가다크펫✨메가앱솔✨메가얼음귀신✨메가보만다✨메가메타그로스✨메가라티아스✨메가라티오스✨원시가이오가✨원시그란돈✨메가레쿠쟈✨메가이어롭✨메가한카리아스✨메가루카리오✨메가눈설왕✨메가엘레이드✨메가다부니\n\n\n거다이맥스 · 다이맥스 전설 배틀\n\n다이맥스 배틀 공략\n\n\n레이드 추천 포켓몬\n\n \n\n포켓몬 GO 레이드 티어, 타입별 사용률, 초보자용 추천 포켓몬 (포켓몬 순위)\n\n포켓몬 GO 추천 기술 (졸업 기술) : 추천 포켓몬, 기술 순위 (레이드, 트레이너 배틀 · GO 배틀리그)\n  \n\n\n관련글'),
 Document(id='05323798-f8d2-4235-ac5c-6fe108081440', metadata={'source': 'https://poketory.com/pokemongo_current_raid/'}, page_content='전설 레이드(레이드 아워)메가 레이드📅 1/24 ~ 2/6디아루가✨(1/29, 2/5)📅 1/24 ~ 2/6메가 요가램✨📅 6~17러브로스 화신폼(2/12)📅 6~17메가 마기라스✨📅 17~24이벨타르✨, 제르네아스✨(2/19)📅 17~24메가 한카리아스✨📅 2/24 ~ 3/1「하나로의 길 이벤트」주말에 진행되는 그림자 전설 레이드그림자 레지락✨※ 레이드는 당일 10시 변경, 레